### 1. Устанавливаем необходимые библиотеки

In [ ]:
pip install -r requirements.txt

После установки библиотек необходимо перегазгрузить Kernel. Для этого в верхнем меню выберите Kernel -> Restart Kernel

### 2. Указываем необходимые параметры для работы с облаком

In [ ]:
TOKEN = "" # IAM токен, подробнее - https://yandex.cloud/ru/docs/iam/operations/iam-token/create-for-sa
FOLDER_ID = "" # ваш folder_id в облаке 

##### скачиваем сертификат

In [33]:
!mkdir -p /home/jupyter/datasphere/project/.clickhouse && \
wget "https://storage.yandexcloud.net/cloud-certs/CA.pem" \
     --output-document /home/jupyter/datasphere/project/.clickhouse/root.crt && \
chmod 0600 /home/jupyter/datasphere/project/.clickhouse/root.crt

--2024-05-12 14:10:28--  https://storage.yandexcloud.net/cloud-certs/CA.pem
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3579 (3.5K) [application/x-x509-ca-cert]
Saving to: ‘/home/jupyter/datasphere/project/.clickhouse/root.crt’

     0K ...                                                   100% 5.86G=0s

2024-05-12 14:10:28 (5.86 GB/s) - ‘/home/jupyter/datasphere/project/.clickhouse/root.crt’ saved [3579/3579]



### 3. Тестируем подключение к Clickhouse

In [34]:
# импортируем библиотеку
import clickhouse_connect

In [35]:
# указываем параметры подключения
CA = '/home/jupyter/datasphere/project/.clickhouse/root.crt' # если вы не меняли путь при скачивании сертификата 

VERIFY = True
SECURE = True
USER = 'admin'
PASSWORD = ''
HOST = "" # адрес Clickhouse
PORT = 8443

In [36]:
# выполняем запрос - в случае успешного выполнения будет показана текущая версия Clickhouse
with clickhouse_connect.get_client(
            host=HOST, port=PORT, username=USER,
            password=PASSWORD, secure=SECURE, verify=VERIFY, ca_cert=CA) as ch_client:
        print(ch_client.command('SELECT version()'))

24.3.2.23


### 4. Создаем индекс в Clickhouse для поиска релевантных документов 

In [ ]:
# импортируем необходимые библиотеки. В качестве фреймворка используется langchain

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_core.prompts.prompt import PromptTemplate
from langchain_community.llms import YandexGPT
from langchain_community.embeddings.yandex import YandexGPTEmbeddings
from langchain_community.document_loaders import S3DirectoryLoader
from langchain_community.vectorstores import ClickhouseSettings, Clickhouse

In [38]:
# параметры object storage 

S3_PREFIX = ''
S3_BUCKET = ''
S3_URL = 'https://storage.yandexcloud.net'
S3_ACCESS_KEY_ID = ''
S3_SECRET_ACCESS_KEY = '' # https://yandex.cloud/en/docs/iam/operations/sa/create-access-key

# Создаем объект для считывания документов из бакета в S3
loader = S3DirectoryLoader(
        bucket=S3_BUCKET,
        endpoint_url=S3_URL,
        prefix=S3_PREFIX,
        aws_access_key_id=S3_ACCESS_KEY_ID,
        aws_secret_access_key=S3_SECRET_ACCESS_KEY
    )

In [39]:
# Указываем длины фрагмента, на который разбиваются документы

CHUNK_SIZE = 1000
CHUNK_OVERLAP = 100

In [40]:
# Считываем документы и разбиваем на фрагменты

documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE, 
    chunk_overlap=CHUNK_OVERLAP
)
splitted_docs = text_splitter.split_documents(documents)

In [41]:
# Cоздаем объект YandexGPTEmbeddings для построения векторов с помощью YandexGPT
embeddings = YandexGPTEmbeddings(iam_token=TOKEN, folder_id=FOLDER_ID)

In [43]:
# создаем конфиг подключения через langchain к Clickhouse
ch_config = ClickhouseSettings(
        host=HOST, port=PORT, username=USER, password=PASSWORD)

# Строим вектора по документам и добавляем их в базу Clickhouse

docsearch = Clickhouse.from_documents(
        splitted_docs,
        embeddings,
        config=ch_config,
        secure=SECURE,
        verify=VERIFY,
        ca_cert=CA)

Inserting data...: 100%|██████████| 101/101 [00:00<00:00, 986.70it/s]


### 5. Тестируем поиск по векторам для запроса 

###### Могу ли я использовать эти API в pytorch 2.3.0?
###### torch.jit.quantized APIs

###### torch.jit.quantized.quantize_rnn_cell_modules

###### torch.jit.quantized.quantize_rnn_modules
###### torch.jit.quantized.quantize_linear_modules

###### torch.jit.quantized.QuantizedLinear
###### torch.jit.QuantizedLinearFP16

###### torch.jit.quantized.QuantizedGRU
###### torch.jit.quantized.QuantizedGRUCell
###### torch.jit.quantized.QuantizedLSTM
###### torch.jit.quantized.QuantizedLSTMCell

In [15]:
# Тестируем сохраненные вектора
query = """Могу ли я использовать эти API в pytorch 2.3.0?
    # torch.jit.quantized APIs

    torch.jit.quantized.quantize_rnn_cell_modules

    torch.jit.quantized.quantize_rnn_modules
    torch.jit.quantized.quantize_linear_modules

    torch.jit.quantized.QuantizedLinear
    torch.jit.QuantizedLinearFP16

    torch.jit.quantized.QuantizedGRU
    torch.jit.quantized.QuantizedGRUCell
    torch.jit.quantized.QuantizedLSTM
    torch.jit.quantized.QuantizedLSTMCell
"""
docs = docsearch.similarity_search(query, k=2)

In [64]:
for doc in docs:
    beautify_string = bytes(doc.page_content, "utf-8").decode("unicode_escape")
    print(beautify_string)
    print("\n ----------END_OF_DOC---------- \n")

pytorch
2.2.2
2.3.0
2024-04-26
Backwards Incompatible Changes

Remove deprecated torch.jit.quantized APIs (#118406)
All functions and classes under torch.jit.quantized will now raise an error if called/instantiated. This API has long been deprecated in favor of torch.ao.nn.quantized.

2.2	2.3
# torch.jit.quantized APIs

torch.jit.quantized.quantize_rnn_cell_modules

torch.jit.quantized.quantize_rnn_modules
torch.jit.quantized.quantize_linear_modules

torch.jit.quantized.QuantizedLinear
torch.jit.QuantizedLinearFP16

torch.jit.quantized.QuantizedGRU
torch.jit.quantized.QuantizedGRUCell
torch.jit.quantized.QuantizedLSTM
torch.jit.quantized.QuantizedLSTMCell
# Corresponding torch.ao.quantization APIs

torch.ao.nn.quantized.dynamic.RNNCell

torch.ao.quantization.quantize_dynamic APIs

torch.ao.nn.quantized.dynamic.Linear

torch.ao.nn.quantized.dynamic.GRU
torch.ao.nn.quantized.dynamic.GRUCell
torch.ao.nn.quantized.dynamic.LSTM

 ----------END_OF_DOC---------- 

pytorch
2.2.2
2.3.0
2024-04-

### 6. Отправляем запрос к YaGPT

In [68]:
prompt_template = "Ответь на вопрос {question}"

prompt = PromptTemplate(
    input_variables=["question"],
    template=prompt_template
)

llm = YandexGPT(iam_token=TOKEN, folder_id=FOLDER_ID)
chain = prompt | llm

response = chain.invoke(query)

print(response)

Чтобы узнать, поддерживает ли PyTorch 2.3.0 указанные API, вы можете обратиться к документации PyTorch или документации по конкретной функции.

Документация PyTorch находится по адресу <https://pytorch.org/docs/stable/index.html>.

Документация по API `torch.jit.quantized` находится по адресу <https://pytorch-ru.readthedocs.io/ru/latest/generated/torch.jit.quantized.__index__.htmlmodule-torch.jit.quantized>

Из приведённого вами списка, PyTorch 2.3.0 поддерживает следующие API:

* `torch.jit.quantize\_rnn\_cell\_modules`
* `torch.jit.quantize\_rnn\_modules` 
* `torch.jit.quantize\_linear\_modules`

**torch.jit.quantized.**`QuantizedLinear`:
* **torch**.jit.quantized.*`QuantizedLinearFP16`:
* `QuantizedLinear(in_features, out_features, bias=True, bias_init=None, dtype=None, qscheme=None)`

**torch**.jit.quantized.*`torch.quantized**.`QuantizedGRU`
* torch.quantized.`QuantizedGRU(input_size, hidden_size, num_layers, dropout=0., bias=False, batch_first=False, bidirectional=False, qscheme=

### 7. Отправляем запрос к YaGPT с использованием найденных раннее релевантных документов

In [67]:
# Промпт для обработки документов
document_prompt = PromptTemplate(
    input_variables=["page_content"],
    template="{page_content}"
)

# Промпт для языковой модели
document_variable_name = "context"
stuff_prompt_override = """
    Прими во внимание приложенные к вопросу тексты и ответь на вопрос.
    Текст:
    -----
    {context}
    -----
    Вопрос:
    {query}
    """
prompt = PromptTemplate(
    template=stuff_prompt_override,
    input_variables=["context", "query"]
)

llm = YandexGPT(iam_token=TOKEN, folder_id=FOLDER_ID)

# Создаём цепочку
llm_chain = LLMChain(llm=llm, prompt=prompt)
chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
)

response = chain.invoke({'query': query,
                         'input_documents': docs})

print(response["output_text"])

На основе приведённого вами фрагмента можно сделать вывод, что некоторые функции и классы из пакета **torch.jit.quantized** больше не поддерживаются в версии PyTorch 2.3.0.

Однако более точную информацию можно получить только при изучении полного текста изменений в документации PyTorch версии 2.3.0:  
https://pytorch.org/2.3/changelog/2.3.html

Кроме того, в приведённом вами тексте упоминаются ещё два пакета: **torch.ao.nn** и **torch.ao.quantization**, которые также могут содержать информацию об изменениях, касающихся затронутых функций и классов.  

Если вы хотите узнать, какие именно изменения произошли в PyTorch, вам следует обратиться к документации для этой версии.
